In [1]:
import pandas as pd
import pandas_profiling
import os
import pickle
import gc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb

import optuna

#データ読み込み
train = pd.read_csv("data_EDA/train.csv")
test = pd.read_csv("data_EDA/test.csv")

ModuleNotFoundError: No module named 'optuna'

In [ ]:
# EDA後の特徴量
X_train = train[['DiabetesPedigreeFunction',
                 'BMI',
                 'Glucose',
                 'Age',
                 'Pregnancies',
                 'Pregnancies_bin',
                 'BloodPressure_na' ,
                 'BloodPressure_0', 
                 'SkinThickness',
                 'Insulin_0',
                 'Insulin_na',
                 
                 
                ]]
id_train = train[['index']]
y_train = train[['Outcome']]

In [ ]:
# 探索しないパラメータ

params_base = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metrics': 'auc',
    'learning_rate': 0.02,
    'n_estimators': 100000,
    'bagging_freq': 1,
    'seed': 123,
}
random_state=123

def objective(trial):
    # 探索するパラメータ
    params_tuning = {
        'num_leaves': trial.suggest_int('num_leaves',8,256),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',5,200),
        'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 1e-5, 1e-2, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 1e2, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 1e2, log=True),
    }
    params_tuning.update(params_base)
    
    # モデル学習・評価
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state).split(X_train, y_train))
    for nfold in np.arange(4):
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = X_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
        x_va, y_va = X_train.loc[idx_va, :], y_train.loc[idx_va, :]
        model = lgb.LGBMClassifier(**params_tuning)
        model.fit(x_tr,
                  y_tr,
                  eval_set=[(x_tr, y_tr), (x_va, y_va)],
                  early_stopping_rounds=100,
                  verbose=0,
                 )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = accuracy_score(y_va, np.where(y_va_pred>0.5, 1, 0))
        list_metrics.append(metric_va)
        
    # 評価値の計算
    metrics = np.mean(list_metrics)
    
    return metrics

In [25]:
sampler = optuna.samplers.TPESampler(seed=random_state)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=30)

[I 2023-03-13 15:09:23,293] A new study created in memory with name: no-name-485b408d-5441-4e5a-954d-5a8b0201c9b0


[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.792414358623587e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=4.792414358623587e-05
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] feature_fraction is set=0.7756573845414456, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7756573845414456
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.792414358623587e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_

[I 2023-03-13 15:09:25,775] Trial 0 finished with value: 0.7993333333333333 and parameters: {'num_leaves': 181, 'min_data_in_leaf': 61, 'min_sum_hessian_in_leaf': 4.792414358623587e-05, 'feature_fraction': 0.7756573845414456, 'bagging_fraction': 0.8597344848927815, 'reg_alpha': 0.492522233779106, 'reg_lambda': 83.76388146302445}. Best is trial 0 with value: 0.7993333333333333.


[LightGBM] [Warning] bagging_fraction is set=0.8645248536920208, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8645248536920208
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.00015009027543233888, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.00015009027543233888
[LightGBM] [Warning] min_data_in_leaf is set=99, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=99
[LightGBM] [Warning] feature_fraction is set=0.6715890080754348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6715890080754348
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8645248536920208, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8645248536920208
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.00015009027543233888, min_child_weight=0.001 will be ignored. Current value: min_sum_hessi

[I 2023-03-13 15:09:26,644] Trial 1 finished with value: 0.806 and parameters: {'num_leaves': 178, 'min_data_in_leaf': 99, 'min_sum_hessian_in_leaf': 0.00015009027543233888, 'feature_fraction': 0.6715890080754348, 'bagging_fraction': 0.8645248536920208, 'reg_alpha': 0.567922374174008, 'reg_lambda': 0.01732652966363563}. Best is trial 1 with value: 0.806.


...

[I 2023-03-13 15:09:55,071] Trial 27 finished with value: 0.8083333333333333 and parameters: {'num_leaves': 48, 'min_data_in_leaf': 126, 'min_sum_hessian_in_leaf': 6.972104745468846e-05, 'feature_fraction': 0.591372148877871, 'bagging_fraction': 0.8492100478235491, 'reg_alpha': 1.196667720289308, 'reg_lambda': 0.10114064587525043}. Best is trial 27 with value: 0.8083333333333333.


[LightGBM] [Warning] bagging_fraction is set=0.7882961845828448, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7882961845828448
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=2.2546017139609767e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=2.2546017139609767e-05
[LightGBM] [Warning] min_data_in_leaf is set=129, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=129
[LightGBM] [Warning] feature_fraction is set=0.5864960275913353, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5864960275913353
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.7882961845828448, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7882961845828448
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=2.2546017139609767e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hes

[I 2023-03-13 15:09:56,496] Trial 28 finished with value: 0.8009999999999999 and parameters: {'num_leaves': 44, 'min_data_in_leaf': 129, 'min_sum_hessian_in_leaf': 2.2546017139609767e-05, 'feature_fraction': 0.5864960275913353, 'bagging_fraction': 0.7882961845828448, 'reg_alpha': 1.274718498587182, 'reg_lambda': 0.12755942719839233}. Best is trial 27 with value: 0.8083333333333333.


[LightGBM] [Warning] bagging_fraction is set=0.841048432152907, subsample=1.0 will be ignored. Current value: bagging_fraction=0.841048432152907
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.834522689773493e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=4.834522689773493e-05
[LightGBM] [Warning] min_data_in_leaf is set=162, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=162
[LightGBM] [Warning] feature_fraction is set=0.6055886806527228, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6055886806527228
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.841048432152907, subsample=1.0 will be ignored. Current value: bagging_fraction=0.841048432152907
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.834522689773493e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in

[I 2023-03-13 15:09:57,784] Trial 29 finished with value: 0.804 and parameters: {'num_leaves': 60, 'min_data_in_leaf': 162, 'min_sum_hessian_in_leaf': 4.834522689773493e-05, 'feature_fraction': 0.6055886806527228, 'bagging_fraction': 0.841048432152907, 'reg_alpha': 2.2213318222687475, 'reg_lambda': 0.5724629864564882}. Best is trial 27 with value: 0.8083333333333333.


In [26]:
trial = study.best_trial
print('acc(best)={:.4f}'.format(trial.value))
display(trial.params)

acc(best)=0.8083


{'num_leaves': 48,
 'min_data_in_leaf': 126,
 'min_sum_hessian_in_leaf': 6.972104745468846e-05,
 'feature_fraction': 0.591372148877871,
 'bagging_fraction': 0.8492100478235491,
 'reg_alpha': 1.196667720289308,
 'reg_lambda': 0.10114064587525043}

In [ ]:
params_best = trial.params
params_best.update(params_base)
display(params_best)